<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>pg_cron</title>
    <style>
        .title {
            color: lightblue;
            font-size: 2.5em;
        }
        .description {
            font-size: 1.2em;
        }
        .description-title {
            color: orange;
            font-weight: bold;
        }
        .version-title {
            color: orange;
            font-weight: bold;
            font-size: 1.2em;
        }
        .version-number {
            color: white;
            font-size: 1.2em;
        }
    </style>
</head>
<body>
    <div class="title">pg_cron</div>
    <div class="description">
        <span class="description-title">Description:</span> pg_cron is a PostgreSQL extension that allows you to run SQL commands on a schedule. It is particularly useful for automating routine database tasks without needing an external job scheduler. With pg_cron, you can schedule regular maintenance tasks, data transformations, backups, monitoring queries, and more directly from within your PostgreSQL database.
    </div>
    <div>
        <span class="version-title">Version:</span> <span class="version-number">1.0</span>
    </div>
</body>
</html>


### TODO
- Find the DB in pg_settings
- Show all pg_settings
-- Raise an error if the extension can't be found


In [4]:

import pandas as pd
import configparser
import importlib
import db_ops
importlib.reload(db_ops)
from db_ops import DB_Ops

# Read the con_str from the Config file
config = configparser.ConfigParser()
config.read_file(open(r'../db_ops.cfg'))
con_str = config.get('con_str', 'PG_TEMBO_DEV1') 

# Create an instance of DB_Ops
db_ops = DB_Ops()
sql = db_ops.get_sql_for_operation(operation_name = 'show_pg_cron_tables_overview')
# print (sql)
df = db_ops.execute_sql_query(con_str, sql)
df

,number_of_rows_in_cron_job,number_of_rows_in_cron_job_run_details,oldest_run_date
0,12,782763,2024-05-17 10:55:00.004107+00:00


In [2]:
sql_job_history = """
SELECT
    j.jobid,
    j.jobname,
    COUNT(jrd.*) AS total_runs,
    SUM(CASE WHEN jrd.status = 'succeeded' THEN 1 ELSE 0 END) AS success_count,
    SUM(CASE WHEN jrd.status = 'failed' THEN 1 ELSE 0 END) AS failure_count,
    ROUND((SUM(CASE WHEN jrd.status = 'succeeded' THEN 1 ELSE 0 END)::DECIMAL / COUNT(jrd.*)) * 100, 2) AS success_percentage,
    ROUND((SUM(CASE WHEN jrd.status = 'failed' THEN 1 ELSE 0 END)::DECIMAL / COUNT(jrd.*)) * 100, 2) AS failure_percentage
FROM
    cron.job j
JOIN
    cron.job_run_details jrd ON j.jobid = jrd.jobid
WHERE
    jrd.start_time >= CURRENT_DATE - INTERVAL '30 days'
GROUP BY
    j.jobid, j.jobname
ORDER BY
    j.jobid;
"""

In [3]:
job_history = """
SELECT
 	jrd.runid ,
    jrd.jobid,
    j.jobname,
    jrd.start_time ,
    jrd.end_time - jrd.start_time  AS duration, 
    jrd.status,
    jrd.return_message
FROM
    cron.job j
JOIN
    cron.job_run_details jrd ON j.jobid = jrd.jobid
WHERE
    jrd.jobid = 9
ORDER BY
    jrd.start_time DESC
LIMIT 100;

"""